# Model training

In [135]:
## import 
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    precision_score,
    root_mean_squared_error,
    classification_report
)
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.inspection import permutation_importance
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


import matplotlib.pyplot as plt 


loading dataframe

In [5]:
df = pd.read_csv('../data/processed/cleaned_cybersecurity_attacks_encoded.csv', index_col= 'Attack_Index')
df.head()


,Packet Length,Anomaly Scores,Attack Type,Severity Level,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Traffic Type_DNS,...,Action Taken_Logged.1,Network Segment_Segment A.1,Network Segment_Segment B.1,Network Segment_Segment C.1,Firewall Logs_Log Data.1,Firewall Logs_No Log Data.1,IDS/IPS Alerts_Alert Data.1,IDS/IPS Alerts_No Alert Data.1,Log Source_Firewall.1,Log Source_Server.1
Attack_Index,,,,,,,,,,,,,,,,,,,,,
0,503,28.67,Malware,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,1,0,1
1,1174,51.50,Malware,1,1,0,0,0,1,0,...,0,0,1,0,1,0,0,1,1,0
2,306,87.42,DDoS,1,0,0,1,1,0,0,...,0,0,0,1,1,0,1,0,1,0
3,385,15.79,Malware,2,0,0,1,0,1,0,...,0,0,1,0,0,1,1,0,1,0
4,1462,0.52,DDoS,1,0,1,0,0,1,1,...,0,0,0,1,0,1,1,0,1,0


## Dataset Splitting

In [8]:
df_train, df_test = train_test_split(df,test_size = 0.2, random_state=42)
df_train.shape, df_test.shape

((32000, 56), (8000, 56))

let see the distribution 

In [11]:
print(df['Severity Level'].mean())
print(df_train['Severity Level'].mean())
print(df_test['Severity Level'].mean())

1.001325
0.9975625
1.016375


### Dividing the target column and feature columns

In [14]:
# list of columns features to be used for training
features = [
    'Packet Length', 
    'Anomaly Scores', 
    'Severity Level', 
    'Protocol_ICMP',
    'Protocol_TCP', 
    'Protocol_UDP', 
    'Packet Type_Control',
    'Packet Type_Data', 
    'Traffic Type_DNS', 
    'Traffic Type_FTP',
    'Traffic Type_HTTP', 
    'Malware Indicators_IoC Detected',
    'Malware Indicators_No IoC Detected', 
    'Alerts/Warnings_Alert Triggered',
    'Alerts/Warnings_No Alert Triggered', 
    'Attack Signature_Known Pattern A', 
    'Attack Signature_Known Pattern B',
    'Action Taken_Blocked', 
    'Action Taken_Ignored',
    'Action Taken_Logged',
    'Network Segment_Segment A', 
    'Network Segment_Segment B',
    'Network Segment_Segment C',
    'Firewall Logs_Log Data',
    'Firewall Logs_No Log Data',
    'IDS/IPS Alerts_Alert Data',
    'IDS/IPS Alerts_No Alert Data',
    'Log Source_Firewall',
    'Log Source_Server',
    'Protocol_ICMP',
    'Protocol_TCP',
    'Protocol_UDP',
    'Packet Type_Control',
    'Packet Type_Data',
    'Traffic Type_DNS',
    'Traffic Type_FTP',
    'Traffic Type_HTTP',
    'Malware Indicators_IoC Detected',
    'Malware Indicators_No IoC Detected',
    'Alerts/Warnings_Alert Triggered',
    'Alerts/Warnings_No Alert Triggered',
    'Attack Signature_Known Pattern A', 
    'Attack Signature_Known Pattern B',
    'Action Taken_Blocked', 
    'Action Taken_Ignored', 'Action Taken_Logged',
    'Network Segment_Segment A', 
    'Network Segment_Segment B',
    'Network Segment_Segment C', 
    'Firewall Logs_Log Data',
    'Firewall Logs_No Log Data', 
    'IDS/IPS Alerts_Alert Data',
    'IDS/IPS Alerts_No Alert Data',
    'Log Source_Firewall',
    'Log Source_Server'
]

target_column = 'attack type'

### get the values of the features columns for the training data


In [17]:
X_train = df_train.loc[:,features].values
y_train = df_train['Attack Type'].values

### get the values of the columns for the test data

In [20]:
x_test = df_test.loc[:,features].values
y_test = df_test['Attack Type'].values

# Testing models 

our problem is a classification problem , we then tried the following classification algorithms :

1) Logistic Regression
2) Decision tree classifier
3) Random Forest Classifier
4) XGboost
5) Naive Bayes
6) SGD classifier 

## Logistic regression 

In [25]:
lr_model = LogisticRegression(random_state=42,max_iter=1500)

In [127]:
# here we train the model on the training data
lr_model.fit(X=X_train, y=y_train)

y_test_predicted = dt_model.predict(x_test)

In [128]:
cf = pd.DataFrame(
    columns=["y_test_DDoS","y_test_Intrusion","y_test_Malware"],index=["y_predicted_DDoS","y_predicted_Intrusion","y_predicted_Malware"]
)

cf.loc[:,:] = confusion_matrix(y_true= y_test,y_pred= y_test_predicted)
cf

,y_test_DDoS,y_test_Intrusion,y_test_Malware
y_predicted_DDoS,861,900,875
y_predicted_Intrusion,879,935,907
y_predicted_Malware,905,848,890


In [131]:
cf/len(y_test)

,y_test_DDoS,y_test_Intrusion,y_test_Malware
y_predicted_DDoS,0.107625,0.1125,0.109375
y_predicted_Intrusion,0.109875,0.116875,0.113375
y_predicted_Malware,0.113125,0.106,0.11125


In [133]:
report =classification_report(y_true=y_test, y_pred=y_test_predicted)
print(report)

              precision    recall  f1-score   support

        DDoS       0.33      0.33      0.33      2636
   Intrusion       0.35      0.34      0.35      2721
     Malware       0.33      0.34      0.33      2643

    accuracy                           0.34      8000
   macro avg       0.34      0.34      0.34      8000
weighted avg       0.34      0.34      0.34      8000



## Decision Tree Classifier

In [121]:
dt_model = DecisionTreeClassifier(random_state=42)

dt_model.fit(X=X_train,y=y_train)

y_test_predicted_dt = dt_model.predict(x_test)

cf_dt = pd.DataFrame(
    columns=["y_test_DDoS","y_test_Intrusion","y_test_Malware"],index=["y_predicted_DDoS","y_predicted_Intrusion","y_predicted_Malware"]
)
cf_dt.loc[:,:] = confusion_matrix(y_true= y_test,y_pred= y_test_predicted_dt)
cf_dt

,y_test_DDoS,y_test_Intrusion,y_test_Malware
y_predicted_DDoS,861,900,875
y_predicted_Intrusion,879,935,907
y_predicted_Malware,905,848,890


In [123]:
report_dt = classification_report(y_pred=y_test_predicted_dt,y_true=y_test)
print(report_dt)

              precision    recall  f1-score   support

        DDoS       0.33      0.33      0.33      2636
   Intrusion       0.35      0.34      0.35      2721
     Malware       0.33      0.34      0.33      2643

    accuracy                           0.34      8000
   macro avg       0.34      0.34      0.34      8000
weighted avg       0.34      0.34      0.34      8000



## Random forest classifier

In [115]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X=X_train,y=y_train)
y_test_predicted_rf = rf_model.predict(x_test)

In [116]:
## confusion matrix for Random Forest
cm = confusion_matrix(y_true=y_test,y_pred=y_test_predicted_rf)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[907 844 885]
 [935 873 913]
 [903 851 889]]


In [117]:
confusion_matrix_df = pd.DataFrame(cm, index=["DDoS", "Intrusion", "Malware"], columns=["DDoS", "Intrusion", "Malware"])
print(confusion_matrix_df)

           DDoS  Intrusion  Malware
DDoS        907        844      885
Intrusion   935        873      913
Malware     903        851      889


In [84]:
report_rf = classification_report(y_true=y_test, y_pred=y_test_predicted_rf)
print(report_rf)

              precision    recall  f1-score   support

        DDoS       0.33      0.34      0.34      2636
   Intrusion       0.34      0.32      0.33      2721
     Malware       0.33      0.34      0.33      2643

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



In [99]:
# Cross-Validation Accuracy
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
print("Cross-Validation Accuracy: ", cv_scores.mean())  
print("Cross-Validation Standard Deviation: ", cv_scores.std())

Cross-Validation Accuracy:  0.33196875
Cross-Validation Standard Deviation:  0.008265964666933917


In [ ]:
# check feature importance
feature_importances = pd.DataFrame({'Feature': features, 'Importance': rf_model.feature_importances_})
print(feature_importances.sort_values(by='Importance', ascending=False))

## naives bayes 

In [ ]:
model = GaussianNB()
model.fit(X=X_train,y=y_train)
y_test_predicted_gnb = model.predict(x_test)
cm_gnb = confusion_matrix(y_true=y_test,y_pred=y_test_predicted_gnb)
cm_gnb_df = pd.DataFrame(cm_gnb, index=["DDoS", "Intrusion", "Malware"], columns=["DDoS", "Intrusion", "Malware"])
print(cm_gnb_df)

In [ ]:
#report for GaussianNB
report_gnb = classification_report(y_true=y_test, y_pred=y_test_predicted_gnb)
print(report_gnb)

In [ ]:

# Cross-Validation Accuracy for GaussianNB  
cv_scores_gnb = cross_val_score(model, X_train, y_train, cv=5)
print("Cross-Validation Accuracy for GaussianNB: ", cv_scores_gnb.mean())

## SGD classifier

In [ ]:
clf =  GradientBoostingClassifier(loss="log_loss", random_state=42)
clf.fit(X=X_train,y=y_train)
y_test_predicted_gbc = clf.predict(x_test)


In [ ]:
report_gbc = classification_report(y_true=y_test, y_pred=y_test_predicted_gbc)
print(report_gbc)   

## Xgboost

In [101]:
# Encode target variable
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Train XGBoost Model
xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)
xgb_model.fit(X=X_train, y=y_train_encoded)
y_test_predicted_xgb = xgb_model.predict(x_test)

# Decode predictions back to original labels
y_test_predicted_xgb_decoded = label_encoder.inverse_transform(y_test_predicted_xgb)

# Confusion matrix
cm_xgb = confusion_matrix(y_true=y_test, y_pred=y_test_predicted_xgb_decoded)
print("Confusion Matrix:\n", cm_xgb)

Confusion Matrix:
 [[946 782 908]
 [979 818 924]
 [987 779 877]]


In [103]:
#report for XGBoost
report_xgb = classification_report(y_true=y_test, y_pred=y_test_predicted_xgb_decoded)
print(report_xgb)

              precision    recall  f1-score   support

        DDoS       0.32      0.36      0.34      2636
   Intrusion       0.34      0.30      0.32      2721
     Malware       0.32      0.33      0.33      2643

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



In [107]:
joblib.dump(xgb_model, "model.pkl")

['model.pkl']

In [113]:
joblib.dump(label_encoder,"label_encoder.pkl")

['label_encoder.pkl']